In [1]:
library("data.table")

Warning message:
"package 'data.table' was built under R version 4.1.2"


In [2]:
# prepare dummy input

var_a <- c(
	1,
	2,
	3,
	4,
	2,
	3,
	1,
	5,
	3,
	4,
	6,
	4,
	3,
	7,
	5,
	6,
	4,
	5
)

var_b <- c(
	3,
	2,
	1,
	3,
	5,
	6,
	5,
	6,
	4,
	7,
	9,
	4,
	3,
	2,
	3,
	1,
	3,
	2
)

slk_from <- c(
	0.000,
	0.010,
	0.020,
	0.030,
	0.040,
	0.050,
	0.060,
	0.070,
	0.080,
	0.090,
	0.100,
	0.110,
	0.120,
	0.130,
	0.140,
	0.150,
	0.160,
	0.170
)

slk_to <- c(
	0.010,
	0.020,
	0.030,
	0.040,
	0.050,
	0.060,
	0.070,
	0.080,
	0.090,
	0.100,
	0.110,
	0.120,
	0.130,
	0.140,
	0.150,
	0.160,
	0.170,
	0.180
)

length <- c(
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010,
	0.010
)

data <- as.data.table(data.frame(var_a, var_b, slk_from, slk_to, length))
#data = data.frame(var_a,var_b,slk_from,slk_to,length)
data

var_a,var_b,slk_from,slk_to,length
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,3,0.00,0.01,0.01
2,2,0.01,0.02,0.01
3,1,0.02,0.03,0.01
4,3,0.03,0.04,0.01
2,5,0.04,0.05,0.01
3,6,0.05,0.06,0.01
1,5,0.06,0.07,0.01
5,6,0.07,0.08,0.01
3,4,0.08,0.09,0.01


In [3]:
var <- c("var_a", "var_b")
data_var <- data[, ..var]

In [7]:
min.length <- 0.020 # min(length)
max.length <- 0.100 # sum(length)

In [12]:
# find the range of indecies in the middle of the sequence
# such that if segmentation were performed at that index
# then the lengh of the segments on either side would be
# would be longer than the minimum length

data_length     <- data[["length"]] # array
cumlength_left  <- cumsum(data_length);
cumlength_left
cumlength_right <- cumsum(rev(data_length))
cumlength_right

[1] 0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.10 0.11 0.12 0.13 0.14 0.15
[16] 0.16 0.17 0.18

[1] 0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.10 0.11 0.12 0.13 0.14 0.15
[16] 0.16 0.17 0.18

In [13]:
k_left  <-                  c(1, which(cumlength_left  <= min.length) + 1)
k_left

k_right <- nrow(data_var) - c(0, which(cumlength_right <= min.length))
k_right

[1] 1 2 3

[1] 18 17 16

In [14]:
# note that negating the indecies has the effect of
# removing those indecies from the list
k <- c(1 : nrow(data_var))[-c(k_left, k_right)]
k

[1]  4  5  6  7  8  9 10 11 12 13 14 15

In principal the `cumq` function computes the Q value of a split in n observations:  Q is a measure of hetrogenity of a given split.

$$
Q = 1 - \frac{N_a \sigma^{2}_{a} + N_b \sigma^{2}_{b}}{N \sigma^{2}}
$$

Where $N_a$ and $\sigma_a$ are the number of samples and standard deviation of the first subset, $N_b$ and $\sigma_b$ the same for the second subset, and $N$ and $\sigma$ the same for the entire dataset.

Internal to the function:

`cum_n` produces a list which is 1 item shorter than data, since there are $n-1$ splits in $n$ observations

The computation of `cqvalue` is difficult to understand. It uses a lot-to-date method for calculating the standard deviation involving ['power sums'](https://mathworld.wolfram.com/PowerSum.html);
From wikipedia;

We can get the lot-to-date variance as
$$
s_j = \sum_{k=1}^N{x^{j}_k}
$$
$$
\sigma^2 
= \left (\frac{\sqrt{N s_2 - s_{1}^2}}{N} \right )^2 
= \frac{N s_2 - s_{1}^2}{N^2}
$$


Substituting into the above we get;

$$
Q = 1 - \frac{
    N_a \frac{N_a s_{2 a} - s_{1 a}^2}{N_a^2} 
    + 
    N_b \frac{N_b s_{2 b} - s_{1 b}^2}{N_b^2}
    }{
        N \frac{N s_{2} - s_{1}^2}{N^2}
    }
$$

or the final formula
$$
Q = 1 - \frac{
    s_{2 a} - \frac{s_{1 a}^2}{N_a}
    + 
    s_{2 b} - \frac{s_{1 b}^2}{N_b}
    }{
        s_2 - \frac{s_{1}^2}{N}
    }
$$

In [10]:
cumq <- function(data){ # debug: use cumq to calculate q values along all segment points
    n                    <- length(data)
    cum_n                <- 1:(n - 1) # reduces length by 1
	stopifnot(length(cum_n) == length(data) - 1)
    data_left            <- data[cum_n]
    data_right           <- rev(data)[cum_n]
    cum_data_left        <- cumsum(data_left)
    cum_data_right       <- rev(cumsum(data_right))
    sumd                 <- sum(data)
    cum_datasquare_left  <- cumsum(data_left * data_left)
    cum_datasquare_right <- rev(cumsum(data_right * data_right))

    cqvalue <- (
        1 
        - (
            (cum_datasquare_left  - cum_data_left  * cum_data_left  / cum_n) +
            (cum_datasquare_right - cum_data_right * cum_data_right / rev(cum_n))
        ) / (sum(data * data) - sumd * sumd / n)
    )

    return(cqvalue)
}

In [73]:
seg2 <- function(data, min.length){
    data_var <- data[, ..var]
    data_length <- data[[length]] # array
    cumlength_left <- cumsum(data_length);
	cumlength.right <- cumsum(rev(data_length))
    k_left <- c(1, which(cumlength_left <= min.length) + 1)
    k_right <- nrow(data_var) - c(0, which(cumlength.right <= min.length))
    k <- c(1:nrow(data_var))[-c(k_left, k_right)]

    qvalue <- matrix(NA, length(k), n_var)
    for (i in 1:n_var){
        qvalue[, i] <- cumq(data_var[[i]])[k - 1] # debug: use cumq function instead of q function
    }
    qvalue <- rowMeans(qvalue)
    maxk <- which(qvalue == max(qvalue)) + max(k_left)
    return(maxk)
  }

In [72]:
qvalue <- matrix(NA, length(k), length(var))
for (i in 1 : length(var)) {
	qvalue[, i] <- cumq(data_var[[i]])[k - 1] # debug: use cumq function instead of q function
}

ERROR: Error in qvalue[, i] <- cumq(data_var[[i]])[k - 1]: replacement has length zero


In [50]:
qvalue <- matrix(NA, length(k), length(var))
i=1

    data_each_var <- data_var[[i]] 
    n                    <- length(data_each_var)
    cum_n                <- 1:(n - 1) # reduces length by 1
    stopifnot(length(cum_n) == length(data_each_var) - 1)
    data_left            <- data_each_var[cum_n]
    data_right           <- rev(data_each_var)[cum_n]
    cum_data_left        <- cumsum(data_left)
    cum_data_right       <- rev(cumsum(data_right))
    sumd                 <- sum(data_each_var)
    cum_datasquare_left  <- cumsum(data_left * data_left)
    cum_datasquare_right <- rev(cumsum(data_right * data_right))

    cqvalue <- (
        1 
        - (
            (cum_datasquare_left  - cum_data_left  * cum_data_left  / cum_n) +
            (cum_datasquare_right - cum_data_right * cum_data_right / rev(cum_n))
        ) / (sum(data_each_var * data_each_var) - sumd * sumd / n)
    )
    qvalue[, i] <- cqvalue[k - 1]
i=2
    data_each_var <- data_var[[i]] 
    n                    <- length(data_each_var)
    cum_n                <- 1:(n - 1) # reduces length by 1
    stopifnot(length(cum_n) == length(data_each_var) - 1)
    data_left            <- data_each_var[cum_n]
    data_right           <- rev(data_each_var)[cum_n]
    cum_data_left        <- cumsum(data_left)
    cum_data_right       <- rev(cumsum(data_right))
    sumd                 <- sum(data_each_var)
    cum_datasquare_left  <- cumsum(data_left * data_left)
    cum_datasquare_right <- rev(cumsum(data_right * data_right))

    cqvalue <- (
        1
        - (
            (cum_datasquare_left  - cum_data_left  * cum_data_left  / cum_n) +
            (cum_datasquare_right - cum_data_right * cum_data_right / rev(cum_n))
        ) / (sum(data_each_var * data_each_var) - sumd * sumd / n)
    )
    qvalue[, i] <- cqvalue[k - 1]
	
qvalue
qvalue <- rowMeans(qvalue)
qvalue

0.2316742,0.1541401274
0.1709761,0.1642402184
0.2675948,0.0941695247
0.2992081,0.0286624204
0.5192455,0.0100090992
0.3896493,0.0003184713
0.4524887,0.0007077141
0.4380090,0.0385350318
0.2717283,0.2323599967
0.2737557,0.2579617834
0.3709711,0.2352768251


[1] 0.1929072 0.1676082 0.1808822 0.1639353 0.2646273 0.1949839 0.2265982
 [8] 0.2382720 0.2520441 0.2658587 0.3031240 0.1603360

In [63]:
maxk <- which(qvalue == max(qvalue)) + max(k_left)
#ss<-c(4,6)
ss<-maxk
ss

k1 <- c(1, ss)
k1

k2 <- c(ss - 1, nrow(data))
k2

ll <- k2 - k1 + 1
ll

cum.ll <- c(0, cumsum(ll))
cum.ll


[1] 14

[1]  1 14

[1] 13 18

[1] 13  5

[1]  0 13 18

In [64]:
segid <- c(rep(1:length(ll), ll))
segid

segdatalist <- split(data, segid)
segdatalist

[1] 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2

var_a,var_b,slk_from,slk_to,length
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,3,0.00,0.01,0.01
2,2,0.01,0.02,0.01
3,1,0.02,0.03,0.01
4,3,0.03,0.04,0.01
2,5,0.04,0.05,0.01
3,6,0.05,0.06,0.01
1,5,0.06,0.07,0.01
5,6,0.07,0.08,0.01
3,4,0.08,0.09,0.01


In [67]:
lengthdata <- as.data.table(cbind(length = data[["length"]], seg.id = segid))
lengthdata
seglength.summary <- lengthdata[, .(sum(length)), by = .(seg.id)]
seglength.summary
seglength <- round(seglength.summary[[2]], digits = 10)
seglength

length,seg.id
<dbl>,<dbl>
0.01,1
0.01,1
0.01,1
0.01,1
0.01,1
0.01,1
0.01,1
0.01,1
0.01,1


seg.id,V1
<dbl>,<dbl>
1,0.13
2,0.05


[1] 0.13 0.05

In [74]:
k <- which(seglength > max.length)
k
while(length(k) > 0){
    sa <- sapply(k, function(x){
      seg2(segdatalist[[x]], min_length) + cum.ll[x]
    })
    ss <- sort(c(ss, sa))

    k1 <- c(1, ss)
    k2 <- c(ss - 1, nrow(data))
    ll <- k2 - k1 + 1
    cum.ll <- c(0, cumsum(ll))
    segid <- c(rep(1:length(ll), ll))
    segdatalist <- split(data, segid)
    lengthdata <- as.data.table(cbind(length = data[[length]], seg.id = segid))
    seglength.summary <- lengthdata[, .(sum(length)), by = .(seg.id)]
    seglength <- round(seglength.summary[[2]], digits = 10)
    k <- which(seglength > max_length)
  }
  # add seg.id
  k1 <- c(1, ss)
  k2 <- c(ss - 1, nrow(data))
  ll <- k2 - k1 + 1
  data$seg.id <- rep(1:length(ll), ll)
  data$seg.point <- 0
  data$seg.point[c(1, ss)] <- 1
  data

[1] 1

ERROR: Error in .subset2(x, i, exact = exact): recursive indexing failed at level 2



In [27]:
cum_n
data_left
data_right
cum_data_left
cum_data_right
sumd
cum_datasquare_left
cum_datasquare_right

[1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17

[1] 1 2 3 4 2 3 1 5 3 4 6 4 3 7 5 6 4

[1] 5 4 6 5 7 3 4 6 4 3 5 1 3 2 4 3 2

[1]  1  3  6 10 12 15 16 21 24 28 34 38 41 48 53 59 63

[1] 67 65 62 58 56 53 52 47 44 40 34 30 27 20 15  9  5

[1] 68

[1]   1   5  14  30  34  43  44  69  78  94 130 146 155 204 229 265 281

[1] 305 301 292 276 272 263 262 237 228 212 176 160 151 102  77  41  25

In [38]:
qvalue <- rowMeans(qvalue)
qvalue
maxk <- which(qvalue == max(qvalue)) + max(k_left)
maxk

[1] 0.1929072 0.1676082 0.1808822 0.1639353 0.2646273 0.1949839 0.2265982
 [8] 0.2382720 0.2520441 0.2658587 0.3031240 0.1603360

[1] 14

In [22]:
qvalue

[1] 0.1443267 0.1929072 0.1676082 0.1808822 0.1639353 0.2646273 0.1949839
 [8] 0.2265982 0.2382720 0.2520441 0.2658587 0.3031240 0.1603360 0.1318212

In [39]:
which(qvalue == max(qvalue))

[1] 11

In [34]:
(
        1 
        - (
            (cum_datasquare_left  - cum_data_left  * cum_data_left  / cum_n) +
            (cum_datasquare_right - cum_data_right * cum_data_right / rev(cum_n))
        ) / (sum(data_each_var * data_each_var) - sumd * sumd / n)
    )

[1] 0.16635614 0.23769796 0.23167421 0.17097608 0.26759485 0.29920814
 [7] 0.51924546 0.38964932 0.45248869 0.43800905 0.27172827 0.27375566
[13] 0.37097111 0.15643180 0.10950226 0.02389706 0.03220655

In [35]:
cqvalue[k - 1]

[1] 0.2316742 0.1709761 0.2675948 0.2992081 0.5192455 0.3896493 0.4524887
 [8] 0.4380090 0.2717283 0.2737557 0.3709711 0.1564318

In [41]:
which(qvalue == max(qvalue)) + max(k_left)

[1] 14

In [42]:
c(1,2,3)-c(2,3)

Warning message in c(1, 2, 3) - c(2, 3):
"longer object length is not a multiple of shorter object length"


[1] -1 -1  1

In [60]:
rep(1:2,c(3,4))

[1] 1 1 1 2 2 2 2

In [75]:
shs <- function(var = "deflection", length = "length", data, range = NULL){
  data <- as.data.table(data)
  n_var <- length(var)

  cumq <- function(data){ # debug: use cumq to calculate q values along all segment points
    n                    <- length(data)
    cum_n                <- 1:(n - 1)
    data_left            <- data[cum_n]
    data_right           <- rev(data)[cum_n]
    cum_data_left        <- cumsum(data_left)
    cum_data_right       <- rev(cumsum(data_right))
    sumd                 <- sum(data)
    cum_datasquare_left  <- cumsum(data_left * data_left)
    cum_datasquare_right <- rev(cumsum(data_right * data_right))

    cqvalue <- 1 - ((cum_datasquare_left - cum_data_left * cum_data_left/cum_n) +
      (cum_datasquare_right - cum_data_right * cum_data_right/rev(cum_n)))/
      (sum(data * data) - sumd * sumd/n)
    return(cqvalue)
  }

  min_length <- range[1]
  max_length <- range[2]

  seg2 <- function(data, min.length){
    data_var <- data[, ..var]
    data_length <- data[[length]] # array
    cumlength_left <- cumsum(data_length);
	cumlength.right <- cumsum(rev(data_length))
    k_left <- c(1, which(cumlength_left <= min.length) + 1)
    k_right <- nrow(data_var) - c(0, which(cumlength.right <= min.length))
    k <- c(1:nrow(data_var))[-c(k_left, k_right)]

    qvalue <- matrix(NA, length(k), n_var)
    for (i in 1:n_var){
        qvalue[, i] <- cumq(data_var[[i]])[k - 1] # debug: use cumq function instead of q function
    }
    qvalue <- rowMeans(qvalue)
    maxk <- which(qvalue == max(qvalue)) + max(k_left)
    return(maxk)
  }
  # first segmentation
  ss <- seg2(data, min_length)
  # following segmentation
  k1 <- c(1, ss)
  k2 <- c(ss - 1, nrow(data))
  ll <- k2 - k1 + 1
  cum.ll <- c(0, cumsum(ll))
  segid <- c(rep(1:length(ll), ll))
  segdatalist <- split(data, segid)

  lengthdata <- as.data.table(cbind(length = data[[length]], seg.id = segid))
  seglength.summary <- lengthdata[, .(sum(length)), by = .(seg.id)]
  seglength <- round(seglength.summary[[2]], digits = 10)

  k <- which(seglength > max_length)

  while(length(k) > 0){
    sa <- sapply(k, function(x){
      seg2(segdatalist[[x]], min_length) + cum.ll[x]
    })
    ss <- sort(c(ss, sa))

    k1 <- c(1, ss)
    k2 <- c(ss - 1, nrow(data))
    ll <- k2 - k1 + 1
    cum.ll <- c(0, cumsum(ll))
    segid <- c(rep(1:length(ll), ll))
    segdatalist <- split(data, segid)
    lengthdata <- as.data.table(cbind(length = data[[length]], seg.id = segid))
    seglength.summary <- lengthdata[, .(sum(length)), by = .(seg.id)]
    seglength <- round(seglength.summary[[2]], digits = 10)
    k <- which(seglength > max_length)
  }
  # add seg.id
  k1 <- c(1, ss)
  k2 <- c(ss - 1, nrow(data))
  ll <- k2 - k1 + 1
  data$seg.id <- rep(1:length(ll), ll)
  data$seg.point <- 0
  data$seg.point[c(1, ss)] <- 1
  return(data)
}

shs(var,"length",data,range=c(0.02,0.1))

var_a,var_b,slk_from,slk_to,length,seg.id,seg.point
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
1,3,0.00,0.01,0.01,1,1
2,2,0.01,0.02,0.01,1,0
3,1,0.02,0.03,0.01,1,0
4,3,0.03,0.04,0.01,1,0
2,5,0.04,0.05,0.01,1,0
3,6,0.05,0.06,0.01,1,0
1,5,0.06,0.07,0.01,1,0
5,6,0.07,0.08,0.01,2,1
3,4,0.08,0.09,0.01,2,0
